<a href="https://colab.research.google.com/github/itayg25/ChangeDetectionPolSar/blob/main/sentinal_1_change_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup for the Code.
You need to authenticate with Google and insert the relevant GEE Project to get access to the Sentinal one Copernicus DB

In [16]:

import time, math
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from datetime import datetime
from scipy.stats import norm, gamma, f, chi2
from IPython.display import display
from ipyleaflet import (Map,DrawControl,TileLayer,
                        MeasureControl,
                        basemaps,basemap_to_tiles,
                        LayersControl, WidgetControl)
from geopy.geocoders import Nominatim


In [17]:
# Setup the GEE Python API.
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library. If your GEE project name is different you have to change it
ee.Initialize(project='ee-itaygersten')

# Config App


In [18]:
# *****************************************
# The sequental change detection algorithm
# *****************************************

def chi2cdf(chi2, df):
    """Calculates Chi square cumulative distribution function for
       df degrees of freedom using the built-in incomplete gamma
       function gammainc().
    """
    return ee.Image(chi2.divide(2)).gammainc(ee.Number(df).divide(2))

def det(im):
    """Calculates determinant of 2x2 diagonal covariance matrix."""
    return im.expression('b(0)*b(1)')

def log_det_sum(im_list, j):
    """Returns log of determinant of the sum of the first j images in im_list."""
    im_ist = ee.List(im_list)
    sumj = ee.ImageCollection(im_list.slice(0, j)).reduce(ee.Reducer.sum())
    return ee.Image(det(sumj)).log()

def log_det(im_list, j):
    """Returns log of the determinant of the jth image in im_list."""
    im = ee.Image(ee.List(im_list).get(j.subtract(1)))
    return ee.Image(det(im)).log()

def pval(im_list, j, m=4.4):
    """Calculates -2logRj for im_list and returns P value and -2mlogRj."""
    im_list = ee.List(im_list)
    j = ee.Number(j)


    m2logRj = (log_det_sum(im_list, j.subtract(1))
               .multiply(j.subtract(1))
               .add(log_det(im_list, j))
               .add(ee.Number(2).multiply(j).multiply(j.log()))
               .subtract(ee.Number(2).multiply(j.subtract(1))
               .multiply(j.subtract(1).log()))
               .subtract(log_det_sum(im_list,j).multiply(j))
               .multiply(-2).multiply(m))

    # correction to simple Wilks approximation
    # pv = ee.Image.constant(1).subtract(chi2cdf(m2logRj, 2))
    one= ee.Number(1)
    rhoj = one.subtract(one.add(one.divide(j.multiply(j.subtract(one)))).divide(6).divide(m))
    omega2j = one.subtract(one.divide(rhoj)).pow(2.0).divide(-2)
    rhojm2logRj = m2logRj.multiply(rhoj)
    pv = ee.Image.constant(1).subtract(chi2cdf(rhojm2logRj,2).add(chi2cdf(rhojm2logRj,6).multiply(omega2j)).subtract(chi2cdf(rhojm2logRj,2).multiply(omega2j)))

    return (pv, m2logRj)

def p_values(im_list,m=4.4):
    """Pre-calculates the P-value array for a list of images."""
    im_list = ee.List(im_list)
    k = im_list.length()

    def ells_map(ell):
        """Arranges calculation of pval for combinations of k and j."""
        ell = ee.Number(ell)
        # Slice the series from k-l+1 to k (image indices start from 0).
        im_list_ell = im_list.slice(k.subtract(ell), k)

        def js_map(j):
            """Applies pval calculation for combinations of k and j."""
            j = ee.Number(j)
            pv1, m2logRj1 = pval(im_list_ell, j)
            return ee.Feature(None, {'pv': pv1, 'm2logRj': m2logRj1})

        # Map over j=2,3,...,l.
        js = ee.List.sequence(2, ell)
        pv_m2logRj = ee.FeatureCollection(js.map(js_map))

        # Calculate m2logQl from collection of m2logRj images.
        m2logQl = ee.ImageCollection(pv_m2logRj.aggregate_array('m2logRj')).sum()

        # correction to simple Wilks approximation
        # pvQl = ee.Image.constant(1).subtract(chi2cdf(m2logQl, ell.subtract(1).multiply(2)))
        one = ee.Number(1)
        f = ell.subtract(1).multiply(2)
        rho = one.subtract(ell.divide(m).subtract(one.divide(ell.multiply(m))).divide(f).divide(3))
        omega2 = f.multiply(one.subtract(one.divide(rho)).pow(2)).divide(-4)
        rhom2logQl = m2logQl.multiply(rho)
        pvQl = ee.Image.constant(1).subtract(chi2cdf(rhom2logQl,f).add(chi2cdf(rhom2logQl,f.add(4)).multiply(omega2)).subtract(chi2cdf(rhom2logQl,f).multiply(omega2)))

        pvs = ee.List(pv_m2logRj.aggregate_array('pv')).add(pvQl)
        return pvs

    # Map over l = k to 2.
    ells = ee.List.sequence(k, 2, -1)
    pv_arr = ells.map(ells_map)

    # Return the P value array ell = k,...,2, j = 2,...,l.
    return pv_arr

def filter_j(current, prev):
    """Calculates change maps; iterates over j indices of pv_arr."""
    pv = ee.Image(current)
    prev = ee.Dictionary(prev)
    pvQ = ee.Image(prev.get('pvQ'))
    i = ee.Number(prev.get('i'))
    cmap = ee.Image(prev.get('cmap'))
    smap = ee.Image(prev.get('smap'))
    fmap = ee.Image(prev.get('fmap'))
    bmap = ee.Image(prev.get('bmap'))
    alpha = ee.Image(prev.get('alpha'))
    j = ee.Number(prev.get('j'))
    cmapj = cmap.multiply(0).add(i.add(j).subtract(1))
    # Check      Rj?            Ql?                  Row i?
    tst = pv.lt(alpha).And(pvQ.lt(alpha)).And(cmap.eq(i.subtract(1)))
    # Then update cmap...
    cmap = cmap.where(tst, cmapj)
    # ...and fmap...
    fmap = fmap.where(tst, fmap.add(1))
    # ...and smap only if in first row.
    smap = ee.Algorithms.If(i.eq(1), smap.where(tst, cmapj), smap)
    # Create bmap band and add it to bmap image.
    idx = i.add(j).subtract(2)
    tmp = bmap.select(idx)
    bname = bmap.bandNames().get(idx)
    tmp = tmp.where(tst, 1)
    tmp = tmp.rename([bname])
    bmap = bmap.addBands(tmp, [bname], True)
    return ee.Dictionary({'i': i, 'j': j.add(1), 'alpha': alpha, 'pvQ': pvQ,
                          'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap':bmap})

def filter_i(current, prev):
    """Arranges calculation of change maps; iterates over row-indices of pv_arr."""
    current = ee.List(current)
    pvs = current.slice(0, -1 )
    pvQ = ee.Image(current.get(-1))
    prev = ee.Dictionary(prev)
    i = ee.Number(prev.get('i'))
    alpha = ee.Image(prev.get('alpha'))
    median = prev.get('median')
    # Filter Ql p value if desired.
    pvQ = ee.Algorithms.If(median, pvQ.focal_median(2.5), pvQ)
    cmap = prev.get('cmap')
    smap = prev.get('smap')
    fmap = prev.get('fmap')
    bmap = prev.get('bmap')
    first = ee.Dictionary({'i': i, 'j': 1, 'alpha': alpha ,'pvQ': pvQ,
                           'cmap': cmap, 'smap': smap, 'fmap': fmap, 'bmap': bmap})
    result = ee.Dictionary(ee.List(pvs).iterate(filter_j, first))
    return ee.Dictionary({'i': i.add(1), 'alpha': alpha, 'median': median,
                          'cmap': result.get('cmap'), 'smap': result.get('smap'),
                          'fmap': result.get('fmap'), 'bmap': result.get('bmap')})

def dmap_iter(current, prev):
    """Reclassifies values in directional change maps."""
    prev = ee.Dictionary(prev)
    j = ee.Number(prev.get('j'))
    image = ee.Image(current)
    avimg = ee.Image(prev.get('avimg'))
    diff = image.subtract(avimg)
    # Get positive/negative definiteness.
    posd = ee.Image(diff.select(0).gt(0).And(det(diff).gt(0)))
    negd = ee.Image(diff.select(0).lt(0).And(det(diff).gt(0)))
    bmap = ee.Image(prev.get('bmap'))
    bmapj = bmap.select(j)
    dmap = ee.Image.constant(ee.List.sequence(1, 3))
    bmapj = bmapj.where(bmapj, dmap.select(2))
    bmapj = bmapj.where(bmapj.And(posd), dmap.select(0))
    bmapj = bmapj.where(bmapj.And(negd), dmap.select(1))
    bmap = bmap.addBands(bmapj, overwrite=True)
    # Update avimg with provisional means.
    i = ee.Image(prev.get('i')).add(1)
    avimg = avimg.add(image.subtract(avimg).divide(i))
    # Reset avimg to current image and set i=1 if change occurred.
    avimg = avimg.where(bmapj, image)
    i = i.where(bmapj, 1)
    return ee.Dictionary({'avimg': avimg, 'bmap': bmap, 'j': j.add(1), 'i': i})

def change_maps(im_list, median=False, alpha=0.01):
    """Calculates thematic change maps."""
    k = im_list.length()
    # Pre-calculate the P value array.
    pv_arr = ee.List(p_values(im_list))
    # Filter P values for change maps.
    cmap = ee.Image(im_list.get(0)).select(0).multiply(0)
    bmap = ee.Image.constant(ee.List.repeat(0,k.subtract(1))).add(cmap)
    alpha = ee.Image.constant(alpha)
    first = ee.Dictionary({'i': 1, 'alpha': alpha, 'median': median,
                           'cmap': cmap, 'smap': cmap, 'fmap': cmap, 'bmap': bmap})
    result = ee.Dictionary(pv_arr.iterate(filter_i, first))
    # Post-process bmap for change direction.
    bmap =  ee.Image(result.get('bmap'))
    avimg = ee.Image(im_list.get(0))
    j = ee.Number(0)
    i = ee.Image.constant(1)
    first = ee.Dictionary({'avimg': avimg, 'bmap': bmap, 'j': j, 'i': i})
    dmap = ee.Dictionary(im_list.slice(1).iterate(dmap_iter, first)).get('bmap')
    return ee.Dictionary(result.set('bmap', dmap))


def tile_aoi(geometry, max_area_km2=100):
    """Tiles the AOI into smaller squares if it's larger than max_area_km2."""
    # Calculate area and check if tiling is necessary
    area_km2 = geometry.area().divide(1e6).getInfo()
    if area_km2 <= max_area_km2:
        return [geometry]

    # Estimate number of tiles needed (sqrt(area/max_area))
    num_tiles_side = math.ceil(math.sqrt(area_km2 / max_area_km2))

    # Get bounds of the geometry
    bounds = geometry.bounds()
    coords = bounds.coordinates().get(0).getInfo()
    min_x = min(coord[0] for coord in coords)
    max_x = max(coord[0] for coord in coords)
    min_y = min(coord[1] for coord in coords)
    max_y = max(coord[1] for coord in coords)

    # Determine tile width and height
    width = (max_x - min_x) / num_tiles_side
    height = (max_y - min_y) / num_tiles_side

    # Generate tiles
    tiles = []
    for i in range(num_tiles_side):
        for j in range(num_tiles_side):
            tile = ee.Geometry.Rectangle([min_x + i * width, min_y + j * height,
                                          min_x + (i + 1) * width, min_y + (j + 1) * height])
            tiles.append(tile)

    return tiles


## Collect Data and UI functions


In [19]:
#@title Widget Interface

# geolocator - not relevent for internal use
poly = None
geolocator = Nominatim(timeout=10,user_agent='tutorial-pt-4.ipynb')


# collect widgets

w_startdate = widgets.Text(
    value='2023-10-01',
    placeholder=' ',
    description='StartDate:',
    disabled=False
)
w_enddate = widgets.Text(
    value=datetime.today().strftime('%Y-%m-%d'),
    placeholder=' ',
    description='EndDate:',
    disabled=False
)

w_location = widgets.Text(
    value=' ',
    placeholder=' ',
    description='',
    disabled=False
)

w_orbitpass = widgets.RadioButtons(
    options=['ASCENDING','DESCENDING'],
    value='ASCENDING',
    description='Pass:',
    disabled=False
)

w_interval = widgets.BoundedIntText(
    min=1,
    value=1,
    description='Interval:',
    disabled=False
)
w_maxfreq = widgets.BoundedIntText(
    min=1,
    value=20,
    description='MaxFreq:',
    disabled=False
)
w_platform = widgets.RadioButtons(
    options=['Both','A','B'],
     value='Both',
    description='Platform:',
    disabled=False
)
w_relativeorbitnumber = widgets.IntText(
    value='0',
    description='RelOrbit:',
    disabled=False
)

w_exportassetsname = widgets.Text(
    value='projects/<username>/assets/<path>',
    placeholder=' ',
    disabled=False
)

# export widgets

w_export_drive_folder = widgets.Text(
    value='GEE_Export',
    placeholder='export drive folder',
    description='export drive folder:',
    disabled=False
)

w_export_file_name = widgets.Text(
    value=f"cmap_export_{datetime.today().strftime('%Y-%m-%d')}",
    placeholder='export drive file name',
    description='file name:',
    disabled=False
)

w_export_scale = widgets.BoundedIntText(
    value=13,
    min=1,
    description='export scale:',
)

w_export_plate = widgets.Checkbox(
    layout = widgets.Layout(width='200px'),
    value=True,
    description='use plate',
    disabled=False
)

w_stride = widgets.BoundedIntText(
    value=1,
    min=1,
    description='Stride:',
    layout = widgets.Layout(width='200px'),
    disabled=False
)
w_median = widgets.Checkbox(
    value=True,
    description='MedianFilter',
    disabled=False
)
w_significance = widgets.BoundedFloatText(
    value='0.005',
    min=0.00001,
    max=1.00,
    step=0.00001,
    description='Signif:',
    disabled=False
)
w_maskchange = widgets.Checkbox(
    value=True,
    description='NCMask',
    disabled=False
)
w_maskwater = widgets.Checkbox(
    value=True,
    description='WaterMask',
    disabled=False
)
w_S2 = widgets.Checkbox(
    layout = widgets.Layout(width='200px'),
    value=False,
    description='ShowS2',
    disabled=False
)
w_opacity = widgets.BoundedFloatText(
    value='1.0',
    min=0.0,
    max=1.0,
    step=0.1,
    description='Opacity:',
    disabled=False
)
w_out = widgets.Output(
    layout=widgets.Layout(border='1px solid black')
)

w_collect = widgets.Button(description="Collect",disabled=True)
w_preview = widgets.Button(description="Preview",disabled=True)
w_review = widgets.Button(description="ReviewAsset",disabled=False)
w_reset = widgets.Button(description='Clear',disabled=False)
w_goto = widgets.Button(description='GoTo',disabled=False)
w_export_ass = widgets.Button(description='ExportToAssets',disabled=True)
w_export_drv = widgets.Button(description='ExportToDrive',disabled=False)
w_plot = widgets.Button(description='PlotAsset',disabled=False)

w_masks = widgets.VBox([w_maskchange,w_maskwater])
w_dates = widgets.VBox([w_startdate,w_enddate])
w_assets = widgets.VBox([w_review,w_plot])
w_bmap = widgets.VBox([w_interval,w_maxfreq])
w_export = widgets.VBox([widgets.HBox([w_export_scale, w_export_plate]),widgets.HBox([w_export_drive_folder, w_export_file_name]),widgets.HBox([w_export_drv])])
w_signif = widgets.VBox([w_significance,w_median])

In [20]:
#@title Widget function
def on_widget_change(b):
    w_preview.disabled = True
    w_export_ass.disabled = True
    w_export_drv.disabled = True

def on_changemap_widget_change(b):
    if b['new']=='Bitemporal':
        w_interval.disabled=False
    else:
        w_interval.disabled=True
    if b['new']=='Frequency':
        w_maxfreq.disabled=False
    else:
        w_maxfreq.disabled=True

def on_reset_button_clicked(b):
    with w_out:
        w_out.clear_output()
        print('Algorithm output')

w_reset.on_click(on_reset_button_clicked)

def on_goto_button_clicked(b):
    try:
        location = geolocator.geocode(w_location.value)
        m.center = (location.latitude,location.longitude)
        m.zoom = 20
    except Exception as e:
        with w_out:
            print('Error: %s'%e)



# @title Widget Functions

def get_incidence_angle(image):
    ''' grab the mean incidence angle '''
    result = ee.Image(image).select('angle') \
           .reduceRegion(ee.Reducer.mean(),geometry=poly,maxPixels=1e9) \
           .get('angle') \
           .getInfo()
    if result is not None:
        return round(result,2)
    else:
        #incomplete overlap, so use all of the image geometry
        return round(ee.Image(image).select('angle') \
           .reduceRegion(ee.Reducer.mean(),maxPixels=1e9) \
           .get('angle') \
           .getInfo(),2)

def GetTileLayerUrl(image):
    map_id = ee.Image(image).getMapId()
    return map_id["tile_fetcher"].url_format

def handle_draw(self, action, geo_json):
    global poly
    coords =  geo_json['geometry']['coordinates']
    if action == 'created':
        poly = ee.Geometry.Polygon(coords)
        w_preview.disabled = True
        w_export_ass.disabled = True
        # w_export_drv.disabled = True
        w_collect.disabled = False
    elif action == 'deleted':
        poly = None
        w_collect.disabled = True
        w_preview.disabled = True
        w_export_ass.disabled = True
        # w_export_drv.disabled = True

def getS1collection():
    s1 =  ee.ImageCollection('COPERNICUS/S1_GRD') \
                      .filterBounds(poly) \
                      .filterDate(ee.Date(w_startdate.value), ee.Date(w_enddate.value)) \
                      .filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV','VH'])) \
                      .filter(ee.Filter.eq('resolution_meters', 10)) \
                      .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                      .filter(ee.Filter.eq('orbitProperties_pass', w_orbitpass.value))
    return s1.filter(ee.Filter.contains(rightValue=poly,leftField='.geo'))

def getS2collection():
    s2 = ee.ImageCollection('COPERNICUS/S2') \
                      .filterBounds(poly) \
                      .filterDate(ee.Date(w_startdate.value),ee.Date(w_enddate.value)) \
                      .sort('CLOUDY_PIXEL_PERCENTAGE',True)
    return s2.filter(ee.Filter.contains(rightValue=poly,leftField='.geo'))

def get_vvvh(image):
    ''' get 'VV' and 'VH' bands from sentinel-1 imageCollection and restore linear signal from db-values '''
    return image.select('VV','VH').multiply(ee.Image.constant(math.log(10.0)/10.0)).exp()

def clipList(current,prev):
    ''' clip a list of images and multiply by ENL'''
    imlist = ee.List(ee.Dictionary(prev).get('imlist'))
    poly = ee.Dictionary(prev).get('poly')
    enl = ee.Number(ee.Dictionary(prev).get('enl'))
    ctr = ee.Number(ee.Dictionary(prev).get('ctr'))
    stride = ee.Number(ee.Dictionary(prev).get('stride'))
    imlist =  ee.Algorithms.If(ctr.mod(stride).eq(0),
        imlist.add(ee.Image(current).multiply(enl).clip(poly)),imlist)
    return ee.Dictionary({'imlist':imlist,'poly':poly,'enl':enl,'ctr':ctr.add(1),'stride':stride})

def on_collect_button_clicked(b):
    ''' Collect a time series from the archive
    '''
    global result, count, timestamplist1, archive_crs
    with w_out:
        try:
            w_out.clear_output()
            print('running on GEE archive COPERNICUS/S1_GRD (please wait for raster overlay) ...')
            collection = getS1collection()
            if w_relativeorbitnumber.value > 0:
                collection = collection.filter(ee.Filter.eq('relativeOrbitNumber_start', int(w_relativeorbitnumber.value)))
            if w_platform.value != 'Both':
                collection = collection.filter(ee.Filter.eq('platform_number', w_platform.value))
            collection = collection.sort('system:time_start')
            acquisition_times = ee.List(collection.aggregate_array('system:time_start')).getInfo()
            count = len(acquisition_times)
            if count<2:
                raise ValueError('Less than 2 images found')
            archive_crs = ee.Image(collection.first()).select(0).projection().crs().getInfo()
            timestamplist = []
            for timestamp in acquisition_times:
                tmp = time.gmtime(int(timestamp)/1000)
                timestamplist.append(time.strftime('%x', tmp))
            # print dates
            print("Collection dates:")
            [print(t) for t in timestamplist]

            #Make timestamps in YYYYMMDD format
            timestamplist = [x.replace('/','') for x in timestamplist]
            timestamplist = ['T20'+x[4:]+x[0:4] for x in timestamplist]
            timestamplist = timestamplist[::int(w_stride.value)]



            #In case of duplicates add running integer
            timestamplist1 = [timestamplist[i] + '_' + str(i+1) for i in range(len(timestamplist))]
            count = len(timestamplist)
            if count<2:
                raise ValueError('Less than 2 images found, decrease stride')

            relativeorbitnumbers = map(int,ee.List(collection.aggregate_array('relativeOrbitNumber_start')).getInfo())
            rons = list(set(relativeorbitnumbers))
            print('Images found: %i, platform: %s'%(count,w_platform.value))
            print('Number of 10m pixels contained: %i'%math.floor(poly.area().getInfo()/100.0))
            print('Acquisition dates: %s to %s'%(str(timestamplist[0]),str(timestamplist[-1])))
            print('Relative orbit numbers: '+str(rons))
            if len(rons)==1:
                mean_incidence = get_incidence_angle(collection.first())
                print('Mean incidence angle: %f'%mean_incidence)
            else:
                mean_incidence = 'undefined'
                print('Mean incidence angle: (select one rel. orbit)')

            pcollection = collection.map(get_vvvh)
            collectionfirst = ee.Image(pcollection.first())
            w_export_scale.value = collectionfirst.projection().nominalScale().getInfo()
            pList = pcollection.toList(500)
            first = ee.Dictionary({'imlist':ee.List([]),'poly':poly,'enl':ee.Number(4.4),'ctr':ee.Number(0),'stride':ee.Number(int(w_stride.value))})
            imList = ee.List(ee.Dictionary(pList.iterate(clipList,first)).get('imlist'))

            #Get a preview as collection mean
            collectionfirst = collection.first().select(0,1).rename('b0','b1')
            percentiles = collectionfirst.reduceRegion(ee.Reducer.percentile([2,98]),geometry=poly,scale=w_export_scale.value,maxPixels=10e9)
            mn = ee.Number(percentiles.get('b0_p2'))
            mx = ee.Number(percentiles.get('b0_p98'))
            S1 = collectionfirst.select(0).visualize(min=mn, max=mx, opacity=w_opacity.value)

            # Get an S2 image from the the same interval
            collection2 = getS2collection()
            count1 = collection2.size().getInfo()

            if count1>0:
                s2_image =  ee.Image(collection2.first()).select(['B2','B3','B4']).clip(poly)
                percentiles = s2_image.reduceRegion(ee.Reducer.percentile([2,98]),scale=w_export_scale.value,maxPixels=10e9)
                mn = percentiles.values(['B2_p2','B3_p2','B4_p2'])
                mx = percentiles.values(['B2_p98','B3_p98','B4_p98'])
                S2 = s2_image.visualize(min=mn,max=mx,opacity=w_opacity.value)
                timestamp = s2_image.get('system:time_start').getInfo()
                timestamp = time.gmtime(int(timestamp)/1000)
                timestamp = time.strftime('%x', timestamp).replace('/','')
                timestamps2 = '20'+timestamp[4:]+timestamp[0:4]
                if w_S2.value:
                    print('Best Sentinel-2 from %s'%timestamps2)
            else:
                if w_S2.value:
                    print('No S2 image found')

            #Run the algorithm ************************************************
            result = change_maps(imList, w_median.value, w_significance.value)
            #******************************************************************
            w_preview.disabled = False
            w_export_ass.disabled = False
            w_export_drv.disabled = False
            #Display preview
            if len(m.layers)>3:
                m.remove_layer(m.layers[3])
            if w_S2.value and count1>0:
                m.add_layer(TileLayer(url=GetTileLayerUrl(S2)))
            else:
                m.add_layer(TileLayer(url=GetTileLayerUrl(S1)))
        except Exception as e:
            print('Error: %s'%e)


#@title Preview
watermask = ee.Image('UMD/hansen/global_forest_change_2015').select('datamask').eq(1)

change_map_layers = []

def on_preview_button_clicked(b):
    ''' Preview all change maps on different layers '''
    global change_map_layers
    change_map_layers = []  # Reset the list every time this function is called

    with w_out:
        try:
            # Define color palettes
            jet = 'black,blue,cyan,yellow,red'
            rcy = 'black,red,cyan,yellow'

            # Clear previous outputs and layers
            w_out.clear_output()
            while len(m.layers) > 3:
                m.remove_layer(m.layers[3])

            print('Preparing change map layers...')

            # Modified function to add a layer and store it in the list
            def add_change_map_layer(image_key, palette, max_value, layer_name):
                image = ee.Image(result.get(image_key)).byte()
                if w_maskwater.value:
                    image = image.updateMask(watermask)
                if w_maskchange.value:
                    image = image.updateMask(image.gt(0))

                layer = TileLayer(url=GetTileLayerUrl(image.visualize(min=0, max=max_value, opacity=w_opacity.value, palette=palette)), name=layer_name)
                m.add_layer(layer)
                change_map_layers.append(layer)  # Store the layer reference

            # Add layers for each change map type
            add_change_map_layer('smap', jet, count, 'First Change')
            add_change_map_layer('cmap', jet, count, 'Last Change')
            add_change_map_layer('fmap', jet, w_maxfreq.value, 'Change Frequency')

            # Add layers for each bitemporal interval
            bmap = ee.Image(result.get('bmap')).byte()
            for i in range(count):
                layer_name = f'Bitemporal {timestamplist1[i-1]} --> {timestamplist1[i]}'
                add_change_map_layer(bmap.select(i-1).clip(poly), rcy, 3, layer_name)

        except Exception as e:
            print('Error: %s' % e)

def on_review_button_clicked(b):
    ''' Examine change maps exported to user's assets
    '''
    with w_out:
        try:
            # Verify existence of asset
            asset = ee.Image(w_exportassetsname.value)
            poly = ee.Geometry.Polygon(ee.Geometry(asset.get('system:footprint')).coordinates())
            center = poly.centroid().coordinates().getInfo()
            center.reverse()
            m.center = center
            bnames = asset.bandNames().getInfo()[3:-2]
            count = len(bnames)

            # Define color palettes
            jet = 'black,blue,cyan,yellow,red'
            rcy = 'black,red,cyan,yellow'
            smap = asset.select('smap').byte()
            cmap = asset.select('cmap').byte()
            fmap = asset.select('fmap').byte()
            bmap = asset.select(list(range(3, count+3)), bnames).byte()

            # Clear existing layers except base layers
            while len(m.layers) > 3:
                m.remove_layer(m.layers[3])

            # Function to add a layer
            def add_layer(image, palette, min_val, max_val, opacity):
                if w_maskwater.value:
                    image = image.updateMask(watermask)
                if w_maskchange.value:
                    image = image.updateMask(image.gt(0))
                layer_url = GetTileLayerUrl(image.visualize(min=min_val, max=max_val, opacity=opacity, palette=palette))
                m.add_layer(TileLayer(url=layer_url))

            # Add layers for each option
            add_layer(smap, jet, 0, count, w_opacity.value)  # 'First' option
            add_layer(cmap, jet, 0, count, w_opacity.value)  # 'Last' option
            add_layer(fmap, jet, 0, w_maxfreq.value, w_opacity.value)  # 'Frequency' option

            # 'Bitemporal' option for each interval
            for i in range(count):
                add_layer(bmap.select(i), rcy, 0, 3, w_opacity.value)

            w_collect.disabled = False

        except Exception as e:
            print('Error: %s' % e)



In [21]:
#@title export to drive function


def export_map_to_drive(image_key='cmap', file_name='change_map', folder='GEE_exports', scale=10, palette='black,blue,cyan,yellow,red', use_plate=True):
    """
    Exports an ee.Image to Google Drive with dynamic visualization based on the image's min and max values.

    Parameters:
    - image_key: str, key to retrieve the ee.Image from the result dictionary.
    - description: str, base filename for the export (date and file format will be appended).
    - folder: str, the name of the Drive folder to export to.
    - scale: int, the resolution in meters.
    - region: ee.Geometry, the region to export. If None, uses the image's full geometry.
    - palette: list, color palette for visualization.
    """
    # Retrieve the image from the result dictionary
    export_image = ee.Image(result.get(image_key))
    if use_plate:
      # Dynamically compute the min and max values of the image within the specified region (or the image's full geometry if none is provided)
      stats = export_image.reduceRegion(reducer=ee.Reducer.minMax(), geometry=export_image.geometry(), scale=scale, maxPixels=1e9)

      # Extract min and max values using getInfo(); be careful with large regions as this might take time or fail
      min_val = 0
      max_val = count

      # Define visualization parameters with dynamic min and max
      vis_params = {
          'min': min_val,
          'max': max_val,
          'palette': palette
      }

      # Apply the visualization parameters
      visualized_image = export_image.visualize(**vis_params)

      # Define export task with the visualized image
      task = ee.batch.Export.image.toDrive(image=visualized_image,
                                          description=file_name,
                                          folder=folder,
                                          scale=scale,
                                          fileFormat='GeoTIFF',
                                          skipEmptyTiles=True)


    else:
      # Define export task with the visualized image
      task = ee.batch.Export.image.toDrive(image=export_image,
                                          description=file_name,
                                          folder=folder,
                                          scale=scale,
                                          fileFormat='GeoTIFF',
                                          skipEmptyTiles=True)
       # Start the export task
    task.start()
    with w_out:
      print(f'Exporting {file_name} with dynamic visualization to Google Drive... Check the "Tasks" tab in GEE.')


def on_export_button_clicked(b):
  with w_out:
    print("exporting to drive... this might take a while")
    print("you can watch the export status at: https://code.earthengine.google.com/tasks")
  export_map_to_drive(image_key='cmap',
                      file_name=w_export_file_name.value,
                      folder=w_export_drive_folder.value,
                      scale=w_export_scale.value,
                      use_plate=w_export_plate.value)

In [22]:
#@title Set the widgets actions
#These widget changes require a new collect
w_goto.on_click(on_goto_button_clicked)
w_orbitpass.observe(on_widget_change,names='value')
w_platform.observe(on_widget_change,names='value')
w_relativeorbitnumber.observe(on_widget_change,names='value')
w_startdate.observe(on_widget_change,names='value')
w_enddate.observe(on_widget_change,names='value')
w_stride.observe(on_widget_change,names='value')
w_median.observe(on_widget_change,names='value')
w_significance.observe(on_widget_change,names='value')
w_export_plate.observe(on_widget_change,names='value')
w_export_drive_folder.observe(on_widget_change,names='value')
w_export_scale.observe(on_widget_change,names='value')
w_export_file_name.observe(on_widget_change,names='value')

In [23]:
#@title set widgets rows and columns

# old line - has export functions
# row2 = widgets.HBox([w_collect,w_signif,w_stride,w_S2,w_export,w_assets],layout=widgets.Layout(border='1px solid black'))

horizontal_layout = widgets.Layout(border='1px solid black', margin='10px')


rows = [
  widgets.HBox([w_dates,w_location, w_goto],layout=horizontal_layout),
 widgets.HBox([w_platform,w_orbitpass,w_relativeorbitnumber],layout=horizontal_layout),
 widgets.HBox([w_collect,w_signif,w_stride],layout=horizontal_layout),
 widgets.HBox([w_preview,w_bmap,w_masks],layout=horizontal_layout),
#  widgets.HBox([w_opacity],layout=horizontal_layout),
  widgets.HBox([],layout=horizontal_layout),
  widgets.HBox([w_export],layout=horizontal_layout),
 widgets.HBox([w_reset],layout=horizontal_layout),
 widgets.HBox([w_out],layout=horizontal_layout)
]

box = widgets.VBox(rows)


In [24]:
#@title set clicks events
w_collect.on_click(on_collect_button_clicked)
w_preview.on_click(on_preview_button_clicked)
w_review.on_click(on_review_button_clicked)
w_export_drv.on_click(on_export_button_clicked)

In [25]:

#@title Run the interface
def run():
    global m, center
    center = [33.0,35.4]

    osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
    ews = basemap_to_tiles(basemaps.Esri.WorldStreetMap)
    ewi = basemap_to_tiles(basemaps.Esri.WorldImagery)


    mc = MeasureControl(position='topright',primary_length_unit = 'kilometers')

    dc = DrawControl(polyline={},circlemarker={})
    dc.rectangle = {"shapeOptions": {"fillColor": "#0000ff","color": "#0000ff","fillOpacity": 0.05}}
    dc.polygon = {"shapeOptions": {"fillColor": "#0000ff","color": "#0000ff","fillOpacity": 0.05}}

    dc.on_draw(handle_draw)

    lc = LayersControl(position='topright')

    m = Map(center=center,
            max_zoom=18,
                    zoom=8,
                    layout={'height':'600px','width':'1200px'},
                    layers=(ewi,ews,osm),
                    controls=(dc,lc,mc))

    # Create a slider widget
    opacity_slider = widgets.FloatSlider(description='Opacity:', min=0, max=1, value=1)

    # Function to update opacity for all change map layers
    def update_opacity(change):
        for layer in change_map_layers:
            layer.opacity = change['new']

    # Link slider value to the update_opacity function
    opacity_slider.observe(update_opacity, 'value')

    # Add slider to the map
    slider_control = WidgetControl(widget=opacity_slider, position='topright')
    m.add_control(slider_control)

    with w_out:
        w_out.clear_output()
        print('Algorithm output')
    display(m)
    return box



---



# Application


---


### How to run:

1. Set dates
2. Press collect
3. press preview



In [26]:
run()

Map(center=[33.0, 35.4], controls=(DrawControl(options=['position'], polygon={'shapeOptions': {'fillColor': '#…

# Docs


## How to use the interface

### Caveat

*The sequential SAR change detection method that we developed in the first three parts of this tutorial is _pixel-oriented_. That is to say, it is based entirely on the statistical properties of each and every multi-look pixel in the observed image series. For this reason it is best to limit your analysis to a region less than a few thousand square kilometers while interactively visualizing and exploring results using this notebook (for larger regions, [export the results](https://developers.google.com/earth-engine/guides/python_install#exporting-data)). Furthermore, since we are analysing time series for changes, the reflectance images involved must all completely overlap the region of interest and have been irradiated from the same position in space. This means choosing either ascending **or** descending node, and only **one** relative orbit number for any given sequence.*

### Walk through

To get started, let's see how to generate a small change map. In the widget interface above, choose **Platform** A, leaving all other settings as is. Select a small area of interest (aoi) near the town of Jülich with the rectangular or polygon **draw tool**. This will enable the **Collect** button. Click it to collect an image series, the details of which are printed in the info box at the bottom. The raster overlay shows the complete swath of the last image in the sequence. When the overlay is fully rendered, the **Preview** button is enabled.

**Note:** Depending on the position of the aoi, two relative orbit numbers may be displayed (88 and 15). If so, in the corresponding **RelOrbit** field choose either of them and re-collect.

The **Preview** button will  now trigger the change detection algorithm at the scale selected by the current zoom setting. The color coded change map is displayed, showing, for each pixel, the interval within the series of the **First** detected change (palette = 'black, blue, cyan, yellow, red' indicating early in the series through to late). The map displayed is set by the Radio Button next to **Preview**.  Since processing is carried out in parallel, the change image is built up tile-by-tile. As explained in [Part 3](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3#a_question_of_scale)  of this tutorial, the zoom setting can falsify the result somewhat, depending on the pyramid level at which the calculation is carried out. Nevertheless it is often convenient for generating a quick overview. You can see the effect by zooming in and out. De-select the **QuickPreview** check box to override it. Now the calculation is carried out at the full 10 m pixel scale irrespective of the zoom level chosen, but can take considerably longer.

If and when you are satisfied with the previewed result, you can export the change maps to your GEE cloud assets with the **ExportToAssets** button, see below.

### The widgets

**Platform:** Choose one or both of the Sentinel-1 satellites.

**Pass:** Choose ascending or descending node.

**RelOrbit:** Choose relative orbit number. If set to 0 all orbit numbers are included with images which overlap with the area of interest.

**StartDate:** Beginning of collected time series.

**EndDate:** End of collected time series.

**Collect:** Start collection, enabled when an area of interest has been chosen. Upon completion the last Sentinel-1 image in the sequence is displayed.

**Signif:** Choose a significance level for the [likelihood ratio test](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2#the_likelihood_ratio_test).

**MedianFilter:** Run a 5x5 median filter over the change map before displaying or exporting.

**Stride:** Select only a subset of the collected sequence. For example, the value 3 will collect every third image in the sequence.

**ShowS2:** Display the most cloud-free Sentinel-2 image found within the chosen time period instead of the last Sentinel-1 image.

**ExportToAssets:** Creates and runs a batch task to export a change map image as a raster to an Earth Engine asset.  For a time series of $k$ images, the exported change map consists of $k+2$ bands
- cmap: the interval* of the most recent change, one band, byte values $\in [0,k-1]$, where 0 = no change.
- smap: the interval of the first change, one band, byte values $\in [0,k-1]$, where 0 = no change.
- fmap: the number of changes, one band, byte values $\in [0,k-1]$, where 0 = no changes.
- bmap: the changes in each interval, $\ k-1$ bands, byte values $\in [0,3]$, where 0 = no change, 1 = positive definite change, 2 = negative definite change, 3 = indefinite change.

*Two successive acquisition times in the series.

**ExportToDrive:** Sends the change map described above to Drive storage in GeoTIFF format.

**Preview:** Run the change detection algorithm and preview results according to the chosen settings (often slow, depending upon series length, zoom level and size of the aoi).

**ReviewAsset:** Display a currently selected change map asset according to the chosen settings (very fast, since calculations have already been performed).

**PlotAsset:** Plot the proportion of change pixels in the bmap bands of the selected asset as a function of time.

**Bitemporal:** Preview (or ReviewAsset) the change map for one interval of the series (black = no change, red = positive definite, cyan = negative definite, yellow = indefinite).

**Interval:** Choose the interval for the Bitemporal map.

**First:** Preview (or ReviewAsset) the smap band (palette = 'black, blue, cyan, yellow, red' indicating no change, early in the series through to late).

**Last:** Preview (or ReviewAsset) the cmap band (palette = 'black, blue, cyan, yellow, red' indicating no change, early in the series through to late).

**Frequency:** Preview (or ReviewAsset) the fmap band (palette = 'black, blue, cyan, yellow, red' indicating no change, few changes through to many).

**MaxFreq:** The number of changes in the frequency map which corresponds to  'red' or 'many'.

**NCMask:** Mask out (make transparent) the no change pixels in the Preview (or ReviewAsset) overlays.

**WaterMask:** Mask out (make transparent) the water pixels in the Preview (or ReviewAsset) overlays.

**QuickPreview:** When set, calculate the Preview at the pyramid level corresponding to the current zoom level. Otherwise use the native scale of 10 m.

**Opacity:** Set the opacity in the Preview (or ReviewAsset) overlays.

**Clear:** Clear the output window.

**GoTo:** Jump to a geographic location.